In [ ]:
import time
import numpy as np

#유틸


In [ ]:
epsilon = 0.0001

def _t(x):
  return np.transpose(x)

def _m(A, B):
  return np.matmul(A,B)

def sigmoid(x):
  return 1/(1+np.exp(-x))
def mean_squared_error(h, y):
  return 1/2*np.mean(np.square(h-y))

#뉴런 구현

In [ ]:
class Neuron:
    def __init__(self, W, b, a):
    #parameter
        self.W = W
        self.b = b
        self.a = a
    #gradients를 저장하기 위해
        self.dW = np.zeros_like((self.W))
        self.db = np.zeros_like((self.b))
    def __call__(self, x):
        return self.a(_m(_t(self.W),x) + self.b) #W^Tx + b 니까, W가 아직 transpose전이니까


#심층 신경망 구현

In [ ]:
class DNN:
  def __init__(self,hidden_depth, num_neuron, num_input, num_output, activation = sigmoid):
    def init_var(i, o):#input과 output의 뉴런의 개수
      return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,))#뒤는 bias

    self.sequence = list()
    #1st
    W, b = init_var(num_input,num_neuron)
    self.sequence.append(Neuron(W,b,activation))

    #other hidden
    for _ in range(hidden_depth - 1):
      W, b = init_var(num_neuron, num_neuron)
      self.sequence.append(Neuron(W,b,activation))

    #output
    W, b = init_var(num_neuron,num_output)
    self.sequence.append(Neuron(W,b,activation))

  def __call__(self,x):
    for layer in self.sequence:#시퀀스의 레이어를 호출해서 반복
      x = layer(x)
    return x

  def calc_gradient(self,x,y,loss_func):#x는 입력, y는 정답
    #gradient 구현
    #핵심!!
    def get_new_sequence(layer_index, new_neuron):
      new_sequence = list()
      for i, layer in enumerate(self.sequence):
        if i == layer_index:
          new_sequence.append(new_neuron)
        else:
          new_sequence.append(layer)
      return new_sequence
      
    def eval_sequence(x, sequence):
      for layer in sequence:
        x = layer(x)
      return x
    
    loss = loss_func(self(x), y)#self는 dnn이라 self.__init__(x)임
    
    for layer_id, layer in enumerate(self.sequence):#iterate layer
      for w_i, w in enumerate(layer.W):#iterate W(row)
        for w_j, ww in enumerate(w):#iterate W(col)
          W = np.copy(layer.W)
          W[w_i][w_j] = ww+epsilon

          new_seq = get_new_sequence(layer_id,Neuron(W,layer.b,layer.a))
          h = eval_sequence(x,new_seq)#하나만 변경한 상태에ㅔ서 평가

          num_grad = (loss_func(h,y)-loss)/epsilon#new eval과 정답을 넣은 loss다시 계산
          #class neuron에 d뭐시기로 다 저장시킨다.
          layer.dW[w_i][w_j] = num_grad
          
        for b_i, bb in enumerate(layer.b):
          b = np.copy(layer.b)
          b[b_i] = bb +epsilon

          new_seq = get_new_sequence(layer_id,Neuron(layer.W,b,layer.a))
          h = eval_sequence(x,new_seq)

          num_grad = (loss_func(h,y)-loss)/epsilon
          layer.db[b_i] = num_grad

    return loss#loss확인 시키기 

#경사 하강

In [ ]:
def gradient_descent(network,x,y,loss_obj, alpha = 0.01):
  loss = network.calc_gradient(x,y,loss_obj)
  for layer in network.sequence:
    layer.W += -alpha * layer.dW
    layer.b += -alpha * layer.db
  return loss

#실행

In [ ]:
x = np.random.normal(0.0,1.0,(10,))
y = np.random.normal(0.0,1.0,(2,))

dnn = DNN(hidden_depth=5,num_neuron=32,num_input=10,num_output=2,activation=sigmoid)

t = time.time()
for epoch in range(100):
  loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
  print('epoch {}, test loss {}'.format(epoch, loss))
print('{} seconds elapsed'.format(time.time()-t))

epoch 0, test loss 0.056849490790312186
epoch 1, test loss 0.056530615107248376
epoch 2, test loss 0.05621351606872431
epoch 3, test loss 0.055898190543064864
epoch 4, test loss 0.05558463524694437
epoch 5, test loss 0.05527284674826616
epoch 6, test loss 0.05496282146905161
epoch 7, test loss 0.054654555688315565
epoch 8, test loss 0.054348045544940916
epoch 9, test loss 0.05404328704055891
epoch 10, test loss 0.05374027604240778
epoch 11, test loss 0.05343900828620479
epoch 12, test loss 0.05313947937898594
epoch 13, test loss 0.05284168480195741
epoch 14, test loss 0.05254561991332504
epoch 15, test loss 0.052251279951113234
epoch 16, test loss 0.05195866003597652
epoch 17, test loss 0.05166775517398354
epoch 18, test loss 0.05137856025940098
epoch 19, test loss 0.05109107007745359
epoch 20, test loss 0.05080527930706754
epoch 21, test loss 0.05052118252359794
epoch 22, test loss 0.050238774201533554
epoch 23, test loss 0.04995804871718185
epoch 24, test loss 0.049679000351337765
ep